# **R-Squared Disentangled**

### Import Libraries

In [245]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', 500)

### Import Data

In [246]:
dummy = pd.read_excel('C:/Users/User/Documents/GitHub/Datathon_2023/Dados/crypto_fear_greed_data.xlsx')
dummy

,date,daily_index,sentiment,weekly_mean,montly_mean
0,2023-10-23,53,Neutral,NaN,NaN
1,2023-10-22,53,Neutral,52.857143,NaN
2,2023-10-21,63,Greed,NaN,NaN
3,2023-10-20,53,Neutral,NaN,NaN
4,2023-10-19,52,Neutral,NaN,NaN
...,...,...,...,...,...
2083,2018-02-05,11,Extreme Fear,NaN,NaN
2084,2018-02-04,24,Extreme Fear,27.250000,NaN
2085,2018-02-03,40,Fear,NaN,NaN
2086,2018-02-02,15,Extreme Fear,NaN,NaN


## *R-Squared*

In [247]:
fg = dummy.drop(['sentiment', 'weekly_mean', 'montly_mean'], axis=1).set_index('date').sort_values(by='date', ascending=True)
fg['weekly_mean'] = fg['daily_index'].rolling(7).mean()
fg['sentiment'] = 0

fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

# fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
# fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg.head(10)

,daily_index,weekly_mean,sentiment
date,,,
2018-02-01,30,NaN,0
2018-02-02,15,NaN,0
2018-02-03,40,NaN,0
2018-02-04,24,NaN,0
2018-02-05,11,NaN,0
2018-02-06,8,NaN,0
2018-02-07,36,23.428571,Extreme Fear
2018-02-08,30,23.428571,Extreme Fear
2018-02-09,44,27.571429,Fear


In [248]:
tech = pd.read_excel('C:/Users/User/Documents/GitHub/Datathon_2023/Dados/tech_signals_w.xlsx')
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [249]:
df = tech.merge(fg, how='left', left_on='Date', right_on='date')
df.dropna(inplace=True)
df.drop('daily_index', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-12,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.000000,Fear
155,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,Greed
156,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,Neutral
157,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,Neutral
158,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2023-09-25,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,Neutral
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,Neutral
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,Neutral
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,Neutral


In [250]:
rf = pd.read_excel('C:/Users/User/Documents/GitHub/Datathon_2023/Dados/risk_free_d.xlsx')
rf['r_free_day'] = rf['r_free_day'].ffill()
rf['r_free_w'] = rf['r_free_day'].rolling(7).mean()
rf = rf[['observation_date', 'r_free_w']]
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.head(10)

,observation_date,r_free_w
0,2018-11-06,NaN
1,2018-11-07,NaN
2,2018-11-08,NaN
3,2018-11-09,NaN
4,2018-11-12,NaN
5,2018-11-13,NaN
6,2018-11-14,0.000087
7,2018-11-15,0.000087
8,2018-11-16,0.000087
9,2018-11-19,0.000087


In [251]:
dummies = pd.get_dummies(df['sentiment'])
df = pd.concat([df.drop('sentiment', axis=1), dummies], axis=1)
df['Returns'] = df['Adj Close'].pct_change()
df = df.merge(rf, how='left', left_on='Date', right_on='observation_date').dropna()
df.drop('observation_date', axis=1, inplace=True)
df['risk_premium'] = df['Returns'] - df['r_free_w']
df.head(10)

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,Extreme Fear,Extreme Greed,Fear,Greed,Neutral,Returns,r_free_w,risk_premium
39,2018-11-19,4009.969971,42533970000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.428571,False,False,True,False,False,-0.286931,0.000087,-0.287019
40,2018-11-26,4139.877930,42945276472,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.714286,True,False,False,False,False,0.032396,0.000088,0.032308
41,2018-12-03,3614.234375,38386467708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.285714,True,False,False,False,False,-0.126971,0.000090,-0.127061
42,2018-12-10,3252.839111,29869247431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.142857,True,False,False,False,False,-0.099992,0.000092,-0.100084
43,2018-12-17,3998.980225,46021506218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.428571,True,False,False,False,False,0.229381,0.000091,0.229290
44,2018-12-24,3865.952637,39249734918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.142857,False,False,True,False,False,-0.033265,0.000093,-0.033359
45,2018-12-31,4076.632568,34343716582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.000000,False,False,True,False,False,0.054496,0.000095,0.054401
46,2019-01-07,3552.953125,37523454177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.428571,False,False,True,False,False,-0.128459,0.000095,-0.128554
47,2019-01-14,3601.013672,38588596987,0,0,0,0,0,0,0,0,1,0,0,0,0,0,28.000000,False,False,True,False,False,0.013527,0.000095,0.013432
48,2019-01-21,3583.965820,36949379050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.142857,False,False,True,False,False,-0.004734,0.000094,-0.004829


In [252]:
r2 = {}

for i in range(3, 17):
    y = df['risk_premium'][2:df.shape[0]].values
    x = df.iloc[1:df.shape[0]-1, i].values
    x = sm.add_constant(x)  
    model = sm.OLS(y, x).fit()
    e = model.resid
    Ic = []
    for j in range(19, 24):
        r_squared = 1 - (np.sum(df.iloc[1:df.shape[0]-1, j] * e**2) / np.sum(df.iloc[1:df.shape[0]-1, j] * ((y - np.mean(y))**2)))
        Ic.append(r_squared)
    r2[df.columns[i]] = Ic

In [253]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T 
r2_df.columns = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
r2_df

,Extreme Fear,Fear,Neutral,Greed,Extreme Greed
MA_1_9,0.010421,0.027064,-0.002564,0.003699,0.157035
MA_1_12,0.008842,0.019298,0.002452,-0.004630,0.056726
MA_2_9,0.008919,0.022511,0.001511,-0.020867,0.111103
MA_2_12,0.007866,0.012251,0.001440,0.009359,0.062326
MA_3_9,0.025550,0.023640,0.019178,-0.036046,0.105146
MA_3_12,0.021166,-0.005581,0.018620,0.012295,0.061855
MOM_9,0.031002,0.020446,0.041998,0.006225,0.083760
MOM_12,0.013972,0.011726,-0.007185,0.002799,0.061865
VOL_1_9,0.009529,0.028854,0.034042,-0.011507,0.090754
VOL_1_12,0.008386,0.011429,0.014661,0.001236,0.039550
